# One Run

In [36]:
import pandas as pd
import numpy as np

# --- Data setup ---
# columns
index=["Safety", "Performance", "Manufacturability","Compatibility","Efficiency"]

# rows for each model (In respective order based on the index)
lightgbm = [0.00598, 8.01422, 22.84536, 78.32318594, 21.4742,  ]
catboost  = [0.00702, 0.22848, 112.0747, 78.10722314, 6.6758]
xgboost   = [0.00772, 1.1537, 21.80616, 77.10482265, 6.6122 ]

df = pd.DataFrame(
    data={
        "LightGBM": lightgbm,
        "CatBoost": catboost,
        "XGBoost": xgboost
    },
    index=index
)



In [37]:
# --- Apply normalization per row (Minimization) ---
def normalize_min_row(row):
    max_val = row.max()
    min_val = row.min()
    return 9 * (max_val - row) / (max_val - min_val) + 1


In [38]:
# --- Apply normalization per row (Maximization) ---
def normalize_max_row(row):
    max_val = row.max()
    min_val = row.min()
    return 9 * (row - min_val) / (max_val - min_val) + 1

In [39]:
# get the rows that will be minimized and maximized
min_rows = df.loc[['Safety', 'Manufacturability','Performance','Efficiency']]
max_rows = df.loc[['Compatibility']]

df_max = max_rows.apply(normalize_max_row, axis=1)
df_min = min_rows.apply(normalize_min_row, axis=1)

df_norm = pd.concat([df_min, df_max])
order = ["Safety", "Performance", "Efficiency", "Manufacturability", "Compatibility"]
df_norm = df_norm.reindex(order)

print("\n=== Normalized Data (1–10 scale, by row) ===")
df_norm


=== Normalized Data (1–10 scale, by row) ===


,LightGBM,CatBoost,XGBoost
Safety,10.000000,4.620690,1.000000
Performance,1.000000,10.000000,8.930483
Efficiency,1.000000,9.961486,10.000000
Manufacturability,9.896389,1.000000,10.000000
Compatibility,10.000000,8.404692,1.000000


In [40]:
# --- Generate all permutations of LOI (10, 9, 8, 7, 6) ---
import itertools
# here is the LOIs, change it
LOI_base = [10, 9, 8, 7, 6]
all_combinations = list(itertools.permutations(LOI_base))

print(f"\nTotal LOI combinations: {len(all_combinations)}")


Total LOI combinations: 120


In [41]:
# appending to a table

results = []
for i, combo in enumerate(all_combinations, start=1):
    # Convert LOI to percentage
    total = sum(combo)
    loi_percentages = [x / total for x in combo]
    loi_series = pd.Series(loi_percentages, index=df_norm.index)

    # Weighted calculation
    df_weighted = df_norm.mul(loi_series, axis=0)
    final_scores = df_weighted.sum()

    # Save to CSV
    filename = f"weighted_results_combo_{i}.csv"
    df_weighted.to_csv(filename, index=True)

    # Store results summary
    results.append({
        "Combo": i,
        "LightGBM": final_scores["LightGBM"],
        "CatBoost": final_scores["CatBoost"],
        "XGBoost": final_scores["XGBoost"],
        "LOI": combo,
    })


In [42]:
# --- Save all final scores summary ---
df_results = pd.DataFrame(results)
df_results.to_csv("all_LOI_results_summary.csv", index=False)

print("\nAll LOI combinations processed and saved!")
print(df_results.head())


All LOI combinations processed and saved!
   Combo  LightGBM  CatBoost   XGBoost               LOI
0      1  6.156868  6.833173  6.159359  (10, 9, 8, 7, 6)
1      2  6.159458  7.018291  5.934359  (10, 9, 8, 6, 7)
2      3  6.379278  6.609136  6.159359  (10, 9, 7, 8, 6)
3      4  6.384458  6.979371  5.709359  (10, 9, 7, 6, 8)
4      5  6.604278  6.570216  5.934359  (10, 9, 6, 8, 7)
